# Best X finder - Test for two periods

Veremos cómo funciona el algoritmo en un script distinto puesto que podría haber variables inicializadas en los test del script *two_best_X_finder.ipynb*

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap
from termcolor import colored
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import array
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 15

In [3]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
os.chdir(path)

/home/usuario/Documentos/Github/Proyectos/MLB_HN/Models/Linear_models/Free_agent/Best_X_finder


In [4]:
print("Para modificar el tamaño de todos los gráficos")
from matplotlib import rcParams
rcParams['figure.figsize'] = 15,9

Para modificar el tamaño de todos los gráficos


In [5]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Test

In [6]:
def etl_regression(X_raw, Y_raw, df_copy):
    # Creación de la base de datos a transformar las variables de la regresión
    jugador = 'Jugador'
    YX_raw = [None]*period
    
    for year in range(0, period):
        YX_raw[year] = df_copy[year][[jugador, Y_raw, X_raw]]

    # Ajuste de inflación en los salarios    
    change_inflation_percentage = {2011:20.46, 2012:18.02, 2013:16.32, 2014:14.46,
                                   2015:14.33, 2016:12.90, 2017:10.55, 2018:7.86,
                                   2019:5.99, 2020:4.70, 2021:1}
    for year in range(0, period):
        YX_raw[year][[Y_raw]] = YX_raw[year][[Y_raw]]*(1 + change_inflation_percentage[2011 + year]*0.01)
        YX_raw[year].rename(columns = {Y_raw: Y_raw + '_' + str(2011 + year)}, inplace = True)
        YX_raw[year].rename(columns = {X_raw: X_raw + '_' + str(2011 + year)}, inplace = True)

    # Filtrado de base de datos
    YX_panel_raw = pd.merge(YX_raw[0], YX_raw[1], how = 'outer', on = 'Jugador')

    for year in range(2,period):
        YX_panel_raw = pd.merge(YX_panel_raw, YX_raw[year], how = 'outer', on = 'Jugador')

    YX_panel_raw.drop_duplicates(subset = 'Jugador', inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw.sort_index(axis = 1, inplace = True)

    # Recolocación al principio de Jugador
    player_column = YX_panel_raw.pop('Jugador')
    YX_panel_raw.insert(0, 'Jugador', player_column)

    # Creación de base de datos auxiliares  para remover los NaN
    split_column = 1 + period
    max_nan = 2

    if Y_raw > X_raw:
        X_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]

    nulls_quantity = period - max_nan
    X_drop_etl = X_drop_etl.iloc[X_drop_etl[(X_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    X_index_drop = X_drop_etl.index

    if Y_raw > X_raw:
        Y_drop_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_drop_etl = YX_panel_raw.iloc[:, 1:split_column]
    Y_drop_etl = Y_drop_etl.iloc[Y_drop_etl[(Y_drop_etl.isnull().sum(axis = 1) > nulls_quantity)].index]
    Y_index_drop = Y_drop_etl.index

    # índice de los jugadores que no tienen más de dos observaciones en el periodo de analisis
    index_drop = list(set(X_index_drop).union(set(Y_index_drop)))

    YX_panel_raw.drop(index = index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)
    YX_panel_raw = YX_panel_raw.sort_values(by = 'Jugador', ascending = True)

    # Borremos provicionalmente a los jugadores cuyo estadística máxima es igual a 0
    if Y_raw > X_raw:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, 1:split_column].max(axis = 1)
    else:
        YX_panel_raw[X_raw + '_max'] = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]].max(axis = 1)
    YX_max_0 = YX_panel_raw[YX_panel_raw[X_raw + '_max'] == 0]
    YX_max_0_index_drop = YX_max_0.index
    YX_panel_raw.drop(columns = X_raw + '_max', inplace = True)
    YX_panel_raw.drop(index = YX_max_0_index_drop, inplace = True)
    YX_panel_raw.reset_index(drop = True, inplace = True)

    # Variable X
    if Y_raw > X_raw:
        X_etl = YX_panel_raw.iloc[:, 1:split_column]
    else:
        X_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    max_element = X_etl.max(axis = 1)
    min_element = X_etl.min(axis = 1)
    max_min = (max_element + min_element)/2
    dummy_names = []
    X_names = []
    for year in range(0,period):
        dummy_names.append("I_" + str(2011 + year))
        X_names.append("X_" + str(2011 + year))
    for year in range(0,period):
        X_conditions = [
        (X_etl.iloc[:,year] == max_element) & ~(X_etl.iloc[:,year].isnull()),
        (X_etl.iloc[:,year] != max_element) & ~(X_etl.iloc[:,year].isnull())
        ]
        X_conditions_values = [0,1]
        X_etl[dummy_names[year]] = np.select(X_conditions, X_conditions_values, default = np.nan)
    X_aux = X_etl.iloc[:,:period]
    Dummy = X_etl.iloc[:,period:]
    X_max = X_aux.max(axis = 1)
    X_aux[['Max']] = np.sqrt(X_max)
    for year in range(0,period):    
        X_etl['X_' + str(2011 + year)] = \
        (X_aux[X_raw + '_' + str(2011 + year)]*
        ((-1)**Dummy['I_' + str(2011 + year)]))
    X_etl = X_etl.div(X_aux['Max'].values, axis = 0)
    X_etl = X_etl.iloc[:,2*period:]
    for i in range(0,X_etl.shape[0]):
        row = X_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            X_etl.iloc[i,j] = sustitute[j]

    X_etl.columns = ['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 
                     'X_7', 'X_8', 'X_9', 'X_10', 'X_11']
    X_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    X_player_column = X_etl.pop('Jugador')
    X_etl.insert(0, 'Jugador', X_player_column)

    # Variable Y
    if Y_raw > X_raw:
        Y_etl = YX_panel_raw.iloc[:, split_column:YX_panel_raw.shape[1]]
    else:
        Y_etl = YX_panel_raw.iloc[:, 1:split_column]
    for i in range(0,Y_etl.shape[0]):
        row = Y_etl.iloc[i]
        row_length = row.size
        sustitute = []

        for j in range(row_length):
            if pd.isna(row[j]) != True:
                sustitute.append(row[j])

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl.iloc[i,j] = sustitute[j]

    Y_etl.columns = ['w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 
                     'w_7', 'w_8', 'w_9', 'w_10', 'w_11']
    Y_etl[['Jugador']] = YX_panel_raw[['Jugador']]
    Y_player_column = Y_etl.pop('Jugador')
    Y_etl.insert(0, 'Jugador', Y_player_column)
    Y_etl_copy = Y_etl.copy()
    y_etl_names = Y_etl_copy['Jugador']
    for i in range(0,len(y_etl_names)):
        row = Y_etl_copy.iloc[i]
        row_length = row.size
        k = 0

        sustitute = []

        for j in range(0, row_length):
            if pd.isna(row[j]) != True:
                k = k + 1

        sustitute.append(y_etl_names[i])
        for j in range(1,k - 1):
            sustitute.append(row[j + 1]**0.5 - row[j]**0.5)

        sustitute_length = len(sustitute)

        for k in range(row_length - sustitute_length):
            sustitute.append(np.nan)

        for j in range(row_length):
            Y_etl_copy.iloc[i,j] = sustitute[j]

    Y_etl_copy.columns = ['Jugador', 'Y_1', 'Y_2', 'Y_3', 'Y_4', 'Y_5', 'Y_6', 
                          'Y_7', 'Y_8', 'Y_9', 'Y_10', 'Y_11']
    Y_etl = Y_etl_copy

    # Base de datos
    regression_list = [None]*(period-1)
    for i in range(0,len(regression_list)):
        merge = pd.merge(Y_etl.iloc[:,[0,i+1]], X_etl.iloc[:,[0,i+1]])
        regression_list[i] = merge
        regression_list[i].columns = ['Jugador', 'Y', 'X']
    df_article_regression = regression_list[0]
    for i in range(1,len(regression_list)):
        df_article_regression = pd.concat([df_article_regression, regression_list[i]])
    df_article_regression = df_article_regression.sort_values(by = 'Jugador', ascending = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    df_article_regression.dropna(inplace = True)
    df_article_regression.reset_index(drop = True, inplace = True)
    
    # Construimos la regresion
    Y = df_article_regression['Y'].tolist()
    X = df_article_regression['X'].tolist()
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    
    # Regresamos como argumento al modelo
    return [df_article_regression, model]

## Exploración de datos

Estudiaremos de manera general si hay datos faltantes, así como el tipo de datos que contiene

## Dos periodos

In [7]:
# Años de análisis
starting_year_t_1 = 2011
period = 11
csv = '.csv'
# Directorios
pitchers_path_t_1 = 'ETL_data/Period_t_1/Free_Agent/Pitchers/free_agents_pitchers_'
hitters_path_t_1 = 'ETL_data/Period_t_1/Free_Agent/Hitters/free_agents_batters_'
# Originales:
df_pitchers_t_1 = [None]*period
df_pitchers_t_1_copy = [None]*period
df_hitters_t_1 = [None]*period
df_hitters_t_1_copy = [None]*period

In [8]:
for year in range(0,period):
    df_pitchers_t_1[year] = pd.read_csv(pitchers_path_t_1 + str(starting_year_t_1 + year) + csv)
    df_pitchers_t_1_copy[year] = df_pitchers_t_1[year].copy()
    
    df_hitters_t_1[year] = pd.read_csv(hitters_path_t_1 + str(starting_year_t_1 + year) + csv)
    df_hitters_t_1_copy[year] = df_hitters_t_1[year].copy()
    
df_pitchers_t_1_copy[0] = df_pitchers_t_1_copy[1]
df_hitters_t_1_copy[0] = df_hitters_t_1_copy[1]

In [9]:
list(enumerate(df_pitchers_t_1_copy[5].columns))

[(0, 'Acronimo_t'),
 (1, 'Acronimo_t_1'),
 (2, 'Altura_t'),
 (3, 'Altura_t_1'),
 (4, 'Anio'),
 (5, 'Anio_t'),
 (6, 'Anio_t_1'),
 (7, 'Anios_contrato'),
 (8, 'Bateos_pitcher_2_t'),
 (9, 'Bateos_pitcher_2_t_1'),
 (10, 'Bateos_pitcher_t'),
 (11, 'Bateos_pitcher_t_1'),
 (12, 'Cantidad_agentes_libres_t'),
 (13, 'Cantidad_agentes_libres_t_1'),
 (14, 'Carreras_2_t'),
 (15, 'Carreras_2_t_1'),
 (16, 'Carreras_ganadas_2_t'),
 (17, 'Carreras_ganadas_2_t_1'),
 (18, 'Carreras_ganadas_t'),
 (19, 'Carreras_ganadas_t_1'),
 (20, 'Carreras_t'),
 (21, 'Carreras_t_1'),
 (22, 'Comando_2_t'),
 (23, 'Comando_2_t_1'),
 (24, 'Comando_t'),
 (25, 'Comando_t_1'),
 (26, 'Control_2_t'),
 (27, 'Control_2_t_1'),
 (28, 'Control_t'),
 (29, 'Control_t_1'),
 (30, 'Dominio_2_t'),
 (31, 'Dominio_2_t_1'),
 (32, 'Dominio_t'),
 (33, 'Dominio_t_1'),
 (34, 'ERA_2_t'),
 (35, 'ERA_2_t_1'),
 (36, 'ERA_t'),
 (37, 'ERA_t_1'),
 (38, 'Edad_t'),
 (39, 'Edad_t_1'),
 (40, 'Equipo_anterior'),
 (41, 'Equipo_t'),
 (42, 'Equipo_t_1'),
 (43, 

In [10]:
list(enumerate(df_hitters_t_1_copy[0].columns))

[(0, 'Acronimo_t'),
 (1, 'Acronimo_t_1'),
 (2, 'Altura_t'),
 (3, 'Altura_t_1'),
 (4, 'Anio'),
 (5, 'Anio_t'),
 (6, 'Anio_t_1'),
 (7, 'Anios_contrato'),
 (8, 'At_bats_2_t'),
 (9, 'At_bats_2_t_1'),
 (10, 'At_bats_t'),
 (11, 'At_bats_t_1'),
 (12, 'Bateos_2_t'),
 (13, 'Bateos_2_t_1'),
 (14, 'Bateos_t'),
 (15, 'Bateos_t_1'),
 (16, 'Cantidad_agentes_libres_t'),
 (17, 'Cantidad_agentes_libres_t_1'),
 (18, 'Edad_t'),
 (19, 'Edad_t_1'),
 (20, 'Equipo_anterior'),
 (21, 'Equipo_t'),
 (22, 'Equipo_t_1'),
 (23, 'Home_runs_2_t'),
 (24, 'Home_runs_2_t_1'),
 (25, 'Home_runs_t'),
 (26, 'Home_runs_t_1'),
 (27, 'Juegos_2_t'),
 (28, 'Juegos_2_t_1'),
 (29, 'Juegos_t'),
 (30, 'Juegos_t_1'),
 (31, 'Juegos_totales_t'),
 (32, 'Juegos_totales_t_1'),
 (33, 'Jugador'),
 (34, 'OPS_2_t'),
 (35, 'OPS_2_t_1'),
 (36, 'OPS_t'),
 (37, 'OPS_t_1'),
 (38, 'Pennants_won_t'),
 (39, 'Pennants_won_t_1'),
 (40, 'Peso_t'),
 (41, 'Peso_t_1'),
 (42, 'Playoffs_t'),
 (43, 'Playoffs_t_1'),
 (44, 'Porcentaje_bateo_2_t'),
 (45, 'Porcen

### Pitchers

In [11]:
pitcher_statistics_t_1 = ['Bateos_pitcher_t', 'Carreras_t', 'Carreras_ganadas_t', 'Comando_t', 'Control_t',
                          'Dominio_t', 'ERA_t', 'Inning_pitched_t', 'Juegos_t', 'Juegos_iniciados_t', 
                          'Losses_t', 'Saves_t', 'TVS_t', 'WAR_t', 'WHIP_t', 'Walks_t', 'Wins_t', 
                          'Bateos_pitcher_2_t', 'Carreras_2_t', 'Carreras_ganadas_2_t', 'Comando_2_t', 'Control_2_t',
                          'Dominio_2_t', 'ERA_2_t', 'Inning_pitched_2_t', 'Juegos_2_t', 'Juegos_iniciados_2_t', 
                          'Losses_2_t', 'Saves_2_t', 'TVS_2_t', 'WAR_2_t', 'WHIP_2_t', 'Walks_2_t', 'Wins_2_t',
                          'Bateos_pitcher_t_1', 'Carreras_t_1', 'Carreras_ganadas_t_1', 'Comando_t_1', 'Control_t_1',
                          'Dominio_t_1', 'ERA_t_1', 'Inning_pitched_t_1', 'Juegos_t_1', 'Juegos_iniciados_t_1', 
                          'Losses_t_1', 'Saves_t_1', 'TVS_t_1', 'WAR_t_1', 'WHIP_t_1', 'Walks_t_1', 'Wins_t_1', 
                          'Bateos_pitcher_2_t_1', 'Carreras_2_t_1', 'Carreras_ganadas_2_t_1', 'Comando_2_t_1', 'Control_2_t_1',
                          'Dominio_2_t_1', 'ERA_2_t_1', 'Inning_pitched_2_t_1', 'Juegos_2_t_1', 'Juegos_iniciados_2_t_1', 
                          'Losses_2_t_1', 'Saves_2_t_1', 'TVS_2_t_1', 'WAR_2_t_1', 'WHIP_2_t_1', 'Walks_2_t_1', 'Wins_2_t_1']
Y_pitcher_raw_t_1 = 'ln_Sueldo_t'
pitcher_Statistics_len = len(pitcher_statistics_t_1)
pitcher_model_t_1 = [None]*pitcher_Statistics_len
df_pitcher_model_t_1 = [None]*pitcher_Statistics_len

In [12]:
for statistic in range(0,pitcher_Statistics_len):
    df_pitcher_model_t_1[statistic], pitcher_model_t_1[statistic] = etl_regression(pitcher_statistics_t_1[statistic],
                                                                                   Y_pitcher_raw_t_1,
                                                                                   df_pitchers_t_1_copy)
    
    print(colored(pitcher_statistics_t_1[statistic], "cyan"))
    print(pitcher_model_t_1[statistic].summary())
    print('\n')

Bateos_pitcher_t
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.4488
Date:                Tue, 11 Oct 2022   Prob (F-statistic):              0.503
Time:                        13:57:41   Log-Likelihood:                 275.71
No. Observations:                 373   AIC:                            -547.4
Df Residuals:                     371   BIC:                            -539.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0361      0.006   

## Hitters

Hagamos lo mismo para los *hitters*.

In [13]:
hitter_statistics_t_1 = ['At_bats_t', 'Bateos_t', 'Home_runs_t', 'Juegos_t',
                         'OPS_t', 'Porcentaje_bateo_t', 'Porcetnaje_juegos_t',
                         'RBI_t', 'TVS_t', 'WAR_t', 'At_bats_2_t', 'Bateos_2_t',
                         'Home_runs_2_t', 'Juegos_2_t', 'OPS_2_t',
                         'Porcentaje_bateo_2_t', 'Porcetnaje_juegos_2_t',
                         'RBI_2_t', 'TVS_2_t', 'WAR_2_t',
                         'At_bats_t_1', 'Bateos_t_1', 'Home_runs_t_1', 'Juegos_t_1',
                         'OPS_t_1', 'Porcentaje_bateo_t_1', 'Porcetnaje_juegos_t_1',
                         'RBI_t_1', 'TVS_t_1', 'WAR_t_1', 'At_bats_2_t_1', 'Bateos_2_t_1',
                         'Home_runs_2_t_1', 'Juegos_2_t_1', 'OPS_2_t_1',
                         'Porcentaje_bateo_2_t_1', 'Porcetnaje_juegos_2_t_1',
                         'RBI_2_t_1', 'TVS_2_t_1', 'WAR_2_t_1']
Y_hitter_raw_t_1 = 'ln_Sueldo_t'
hitter_Statistics_len = len(hitter_statistics_t_1)
hitter_model_t_1 = [None]*hitter_Statistics_len
df_hitter_model_t_1 = [None]*hitter_Statistics_len

In [14]:
for statistic in range(0,hitter_Statistics_len):
    df_hitter_model_t_1[statistic], hitter_model_t_1[statistic] = etl_regression(hitter_statistics_t_1[statistic],
                                                                                 Y_hitter_raw_t_1,
                                                                                 df_hitters_t_1_copy)
    
    print(colored(hitter_statistics_t_1[statistic], "cyan"))
    print(hitter_model_t_1[statistic].summary())
    print('\n')

At_bats_t
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     2.940
Date:                Tue, 11 Oct 2022   Prob (F-statistic):             0.0872
Time:                        13:59:20   Log-Likelihood:                 288.00
No. Observations:                 394   AIC:                            -572.0
Df Residuals:                     392   BIC:                            -564.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0476      0.006     -8.00